In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.metrics import accuracy_score, r2_score, mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [2]:
df = pd.read_csv("./data/fifa23_num_clean.csv")
df_mar = pd.read_csv("./data/fifa23_MAR.csv")
df_mcar = pd.read_csv("./data/fifa23_MCAR.csv")
df_mnar = pd.read_csv("./data/fifa23_MNAR.csv")

#DATASETS CON VALORES NULOS IMPUTADOS

#MEAN IMPUTATION
df_mar_mean = pd.read_csv("./data/fifa23_MAR_MEAN.csv")
df_mcar_mean = pd.read_csv("./data/fifa23_MCAR_MEAN.csv")
df_mnar_mean = pd.read_csv("./data/fifa23_MNAR_MEAN.csv")

#MOST FREQUENT IMPUTATION
df_mar_mf = pd.read_csv("./data/fifa23_MAR_MF.csv")
df_mcar_mf = pd.read_csv("./data/fifa23_MCAR_MF.csv")
df_mnar_mf = pd.read_csv("./data/fifa23_MNAR_MF.csv")

#KNN IMPUTATION
df_mar_knn = pd.read_csv("./data/fifa23_MAR_KNN.csv")
df_mcar_knn = pd.read_csv("./data/fifa23_MCAR_KNN.csv")
df_mnar_knn = pd.read_csv("./data/fifa23_MNAR_KNN.csv")

#MICE IMPUTATION
df_mar_mice = pd.read_csv("./data/fifa23_MAR_MICE.csv")
df_mcar_mice = pd.read_csv("./data/fifa23_MCAR_MICE.csv")
df_mnar_mice = pd.read_csv("./data/fifa23_MNAR_MICE.csv")

In [3]:
df.drop(columns={"Value(in Euro)", "Wage(in Euro)"}, inplace=True)
df

,Overall,Potential,Age,Height(in cm),Weight(in kg),TotalStats,BaseStats,Preferred Foot,Weak Foot Rating,Skill Moves,...,LB Rating,CB Rating,RB Rating,GK Rating,Attacking Work Rate_High,Attacking Work Rate_Low,Attacking Work Rate_Medium,Defensive Work Rate_High,Defensive Work Rate_Low,Defensive Work Rate_Medium
0,91,91,35,169,67,2190,452,0,4,4,...,62,53,62,22,0,1,0,0,1,0
1,91,91,34,185,81,2147,455,1,4,4,...,63,58,63,21,0,0,1,0,0,1
2,91,91,33,185,81,2205,458,1,4,4,...,64,63,64,22,1,0,0,0,0,1
3,91,91,31,181,70,2303,483,1,5,4,...,78,72,78,24,1,0,0,1,0,0
4,91,95,23,182,73,2177,470,1,4,5,...,66,57,66,21,1,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18415,47,56,21,174,68,1287,274,1,3,2,...,40,36,40,15,0,0,1,0,0,1
18416,47,57,17,175,60,1289,267,1,3,2,...,49,49,49,15,0,0,1,0,0,1
18417,47,67,18,170,65,1333,277,1,3,2,...,46,42,46,17,1,0,0,0,0,1
18418,47,61,17,178,65,1113,226,1,3,2,...,47,49,47,15,0,0,1,0,0,1


In [4]:
df_mar

,Overall,Potential,Age,Height(in cm),Weight(in kg),TotalStats,BaseStats,Preferred Foot,Weak Foot Rating,Skill Moves,...,LB Rating,CB Rating,RB Rating,GK Rating,Attacking Work Rate_High,Attacking Work Rate_Low,Attacking Work Rate_Medium,Defensive Work Rate_High,Defensive Work Rate_Low,Defensive Work Rate_Medium
0,91,91.0,35.0,169.0,67.0,2190.0,452.0,0.0,4.0,4.0,...,62.0,53.0,62.0,22.0,0.0,1.0,0.0,0.0,1.0,0.0
1,91,91.0,34.0,185.0,81.0,2147.0,455.0,1.0,4.0,4.0,...,63.0,58.0,63.0,21.0,0.0,0.0,1.0,0.0,0.0,1.0
2,91,91.0,33.0,185.0,81.0,2205.0,458.0,1.0,4.0,4.0,...,64.0,63.0,64.0,22.0,1.0,0.0,0.0,0.0,0.0,1.0
3,91,91.0,31.0,181.0,70.0,2303.0,483.0,1.0,5.0,4.0,...,78.0,72.0,78.0,24.0,1.0,0.0,0.0,1.0,0.0,0.0
4,91,95.0,23.0,182.0,73.0,2177.0,470.0,1.0,4.0,5.0,...,66.0,57.0,66.0,21.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18415,47,56.0,21.0,174.0,68.0,1287.0,274.0,1.0,3.0,2.0,...,40.0,36.0,40.0,15.0,0.0,0.0,1.0,0.0,0.0,1.0
18416,47,57.0,17.0,175.0,60.0,1289.0,267.0,1.0,3.0,2.0,...,49.0,49.0,49.0,15.0,0.0,0.0,1.0,0.0,0.0,1.0
18417,47,67.0,18.0,170.0,65.0,1333.0,277.0,1.0,3.0,2.0,...,46.0,42.0,46.0,17.0,1.0,0.0,0.0,0.0,0.0,1.0
18418,47,61.0,17.0,178.0,65.0,1113.0,226.0,1.0,3.0,2.0,...,47.0,49.0,47.0,15.0,0.0,0.0,1.0,0.0,0.0,1.0


In [5]:
df_mcar

,Overall,Potential,Age,Height(in cm),Weight(in kg),TotalStats,BaseStats,Preferred Foot,Weak Foot Rating,Skill Moves,...,LB Rating,CB Rating,RB Rating,GK Rating,Attacking Work Rate_High,Attacking Work Rate_Low,Attacking Work Rate_Medium,Defensive Work Rate_High,Defensive Work Rate_Low,Defensive Work Rate_Medium
0,91,NaN,35.0,169.0,67.0,2190.0,452.0,NaN,4.0,4.0,...,62.0,53.0,NaN,22.0,0.0,1.0,0.0,0.0,1.0,0.0
1,91,91.0,NaN,185.0,81.0,2147.0,455.0,1.0,4.0,4.0,...,63.0,58.0,63.0,21.0,0.0,0.0,1.0,NaN,0.0,1.0
2,91,91.0,33.0,185.0,81.0,2205.0,458.0,1.0,4.0,4.0,...,64.0,63.0,64.0,22.0,1.0,0.0,0.0,0.0,0.0,1.0
3,91,91.0,NaN,181.0,70.0,NaN,483.0,1.0,5.0,4.0,...,NaN,72.0,78.0,NaN,1.0,0.0,0.0,1.0,0.0,0.0
4,91,95.0,23.0,182.0,73.0,2177.0,470.0,NaN,4.0,5.0,...,NaN,57.0,66.0,21.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18415,47,56.0,21.0,174.0,68.0,1287.0,274.0,1.0,NaN,2.0,...,40.0,36.0,40.0,15.0,0.0,0.0,1.0,0.0,0.0,1.0
18416,47,57.0,17.0,175.0,60.0,1289.0,267.0,1.0,3.0,2.0,...,49.0,49.0,49.0,15.0,0.0,0.0,1.0,NaN,0.0,1.0
18417,47,67.0,18.0,170.0,65.0,1333.0,277.0,1.0,3.0,2.0,...,46.0,42.0,46.0,17.0,1.0,0.0,0.0,0.0,NaN,1.0
18418,47,61.0,17.0,NaN,65.0,1113.0,226.0,1.0,3.0,2.0,...,47.0,49.0,47.0,15.0,0.0,0.0,1.0,0.0,0.0,1.0


In [6]:
df_mnar

,Overall,Potential,Age,Height(in cm),Weight(in kg),TotalStats,BaseStats,Preferred Foot,Weak Foot Rating,Skill Moves,...,LB Rating,CB Rating,RB Rating,GK Rating,Attacking Work Rate_High,Attacking Work Rate_Low,Attacking Work Rate_Medium,Defensive Work Rate_High,Defensive Work Rate_Low,Defensive Work Rate_Medium
0,91,0.914894,0.678571,NaN,NaN,0.921442,0.820144,NaN,0.75,0.75,...,0.642857,NaN,0.642857,NaN,NaN,1.0,0.0,0.0,1.0,NaN
1,91,0.914894,0.642857,0.588235,0.571429,0.893754,0.830935,1.0,0.75,0.75,...,0.657143,0.555556,0.657143,NaN,0.0,0.0,1.0,0.0,0.0,1.0
2,91,0.914894,0.607143,NaN,0.571429,0.931101,0.841727,1.0,0.75,0.75,...,0.671429,0.625000,0.671429,NaN,1.0,0.0,0.0,0.0,0.0,1.0
3,91,0.914894,NaN,0.509804,NaN,0.994205,0.931655,1.0,1.00,0.75,...,0.871429,0.750000,0.871429,NaN,1.0,NaN,0.0,1.0,0.0,0.0
4,91,1.000000,NaN,0.529412,NaN,0.913071,0.884892,1.0,0.75,1.00,...,0.700000,0.541667,0.700000,NaN,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18415,47,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.50,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,1.0,NaN,0.0,1.0
18416,47,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,NaN
18417,47,0.404255,NaN,NaN,NaN,0.369607,NaN,1.0,0.50,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,0.0,0.0,0.0,1.0
18418,47,NaN,NaN,NaN,NaN,NaN,NaN,1.0,0.50,NaN,...,0.428571,NaN,NaN,NaN,0.0,0.0,1.0,0.0,0.0,NaN


In [7]:
df_mar_mean

,Overall,Potential,Age,Height(in cm),Weight(in kg),TotalStats,BaseStats,Preferred Foot,Weak Foot Rating,Skill Moves,...,LB Rating,CB Rating,RB Rating,GK Rating,Attacking Work Rate_High,Attacking Work Rate_Low,Attacking Work Rate_Medium,Defensive Work Rate_High,Defensive Work Rate_Low,Defensive Work Rate_Medium
0,91.0,91.000000,35.000000,169.000000,67.000000,2190.000000,452.000000,0.000000,4.000000,4.000000,...,62.000000,53.000000,62.000000,22.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
1,91.0,91.000000,34.000000,185.000000,81.000000,2147.000000,455.000000,1.000000,4.000000,4.000000,...,63.000000,58.000000,63.000000,21.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
2,91.0,91.000000,33.000000,185.000000,81.000000,2205.000000,458.000000,1.000000,4.000000,4.000000,...,64.000000,63.000000,64.000000,22.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
3,91.0,91.000000,31.000000,181.000000,70.000000,2303.000000,483.000000,1.000000,5.000000,4.000000,...,78.000000,72.000000,78.000000,24.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
4,91.0,95.000000,23.000000,182.000000,73.000000,2177.000000,470.000000,1.000000,4.000000,5.000000,...,66.000000,57.000000,66.000000,21.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18415,47.0,56.000000,21.000000,174.000000,68.000000,1287.000000,274.000000,1.000000,3.000000,2.000000,...,40.000000,36.000000,40.000000,15.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
18416,47.0,57.000000,17.000000,175.000000,60.000000,1289.000000,267.000000,1.000000,3.000000,2.000000,...,49.000000,49.000000,49.000000,15.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
18417,47.0,67.000000,18.000000,170.000000,65.000000,1333.000000,277.000000,1.000000,3.000000,2.000000,...,46.000000,42.000000,46.000000,17.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
18418,47.0,61.000000,17.000000,178.000000,65.000000,1113.000000,226.000000,1.000000,3.000000,2.000000,...,47.000000,49.000000,47.000000,15.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000


In [8]:
df_mcar_mean

,Overall,Potential,Age,Height(in cm),Weight(in kg),TotalStats,BaseStats,Preferred Foot,Weak Foot Rating,Skill Moves,...,LB Rating,CB Rating,RB Rating,GK Rating,Attacking Work Rate_High,Attacking Work Rate_Low,Attacking Work Rate_Medium,Defensive Work Rate_High,Defensive Work Rate_Low,Defensive Work Rate_Medium
0,91.0,70.956866,35.00000,169.000000,67.0,2190.000000,452.000000,0.756815,4.000000,4.000000,...,62.000000,53.0,55.649495,22.000000,0.0,1.0,0.0,0.000000,1.00000,0.0
1,91.0,91.000000,25.23095,185.000000,81.0,2147.000000,455.000000,1.000000,4.000000,4.000000,...,63.000000,58.0,63.000000,21.000000,0.0,0.0,1.0,0.182928,0.00000,1.0
2,91.0,91.000000,33.00000,185.000000,81.0,2205.000000,458.000000,1.000000,4.000000,4.000000,...,64.000000,63.0,64.000000,22.000000,1.0,0.0,0.0,0.000000,0.00000,1.0
3,91.0,91.000000,25.23095,181.000000,70.0,1600.807955,483.000000,1.000000,5.000000,4.000000,...,55.560111,72.0,78.000000,23.189612,1.0,0.0,0.0,1.000000,0.00000,0.0
4,91.0,95.000000,23.00000,182.000000,73.0,2177.000000,470.000000,0.756815,4.000000,5.000000,...,55.560111,57.0,66.000000,21.000000,1.0,0.0,0.0,0.000000,1.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18415,47.0,56.000000,21.00000,174.000000,68.0,1287.000000,274.000000,1.000000,2.944863,2.000000,...,40.000000,36.0,40.000000,15.000000,0.0,0.0,1.0,0.000000,0.00000,1.0
18416,47.0,57.000000,17.00000,175.000000,60.0,1289.000000,267.000000,1.000000,3.000000,2.000000,...,49.000000,49.0,49.000000,15.000000,0.0,0.0,1.0,0.182928,0.00000,1.0
18417,47.0,67.000000,18.00000,170.000000,65.0,1333.000000,277.000000,1.000000,3.000000,2.000000,...,46.000000,42.0,46.000000,17.000000,1.0,0.0,0.0,0.000000,0.08364,1.0
18418,47.0,61.000000,17.00000,181.545367,65.0,1113.000000,226.000000,1.000000,3.000000,2.000000,...,47.000000,49.0,47.000000,15.000000,0.0,0.0,1.0,0.000000,0.00000,1.0


In [9]:
df_mnar_mean

,Overall,Potential,Age,Height(in cm),Weight(in kg),TotalStats,BaseStats,Preferred Foot,Weak Foot Rating,Skill Moves,...,LB Rating,CB Rating,RB Rating,GK Rating,Attacking Work Rate_High,Attacking Work Rate_Low,Attacking Work Rate_Medium,Defensive Work Rate_High,Defensive Work Rate_Low,Defensive Work Rate_Medium
0,91.0,0.914894,0.678571,0.600765,0.559538,0.921442,0.820144,1.0,0.750000,0.750000,...,0.642857,0.644271,0.642857,0.696634,0.292488,1.000000,0.0,0.000000,1.0,0.734499
1,91.0,0.914894,0.642857,0.588235,0.571429,0.893754,0.830935,1.0,0.750000,0.750000,...,0.657143,0.555556,0.657143,0.696634,0.000000,0.000000,1.0,0.000000,0.0,1.000000
2,91.0,0.914894,0.607143,0.600765,0.571429,0.931101,0.841727,1.0,0.750000,0.750000,...,0.671429,0.625000,0.671429,0.696634,1.000000,0.000000,0.0,0.000000,0.0,1.000000
3,91.0,0.914894,0.552646,0.509804,0.559538,0.994205,0.931655,1.0,1.000000,0.750000,...,0.871429,0.750000,0.871429,0.696634,1.000000,0.047937,0.0,1.000000,0.0,0.000000
4,91.0,1.000000,0.552646,0.529412,0.559538,0.913071,0.884892,1.0,0.750000,1.000000,...,0.700000,0.541667,0.700000,0.696634,1.000000,0.000000,0.0,0.000000,1.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18415,47.0,0.577141,0.552646,0.600765,0.559538,0.634081,0.583561,1.0,0.500000,0.566919,...,0.657630,0.644271,0.658375,0.696634,0.000000,0.000000,1.0,0.181391,0.0,1.000000
18416,47.0,0.577141,0.552646,0.600765,0.559538,0.634081,0.583561,1.0,0.592232,0.566919,...,0.657630,0.644271,0.658375,0.696634,0.000000,0.000000,1.0,0.000000,0.0,0.734499
18417,47.0,0.404255,0.552646,0.600765,0.559538,0.369607,0.583561,1.0,0.500000,0.566919,...,0.657630,0.644271,0.658375,0.696634,0.292488,0.000000,0.0,0.000000,0.0,1.000000
18418,47.0,0.577141,0.552646,0.600765,0.559538,0.634081,0.583561,1.0,0.500000,0.566919,...,0.428571,0.644271,0.658375,0.696634,0.000000,0.000000,1.0,0.000000,0.0,0.734499


In [10]:
df_mar_mf

,Overall,Potential,Age,Height(in cm),Weight(in kg),TotalStats,BaseStats,Preferred Foot,Weak Foot Rating,Skill Moves,...,LB Rating,CB Rating,RB Rating,GK Rating,Attacking Work Rate_High,Attacking Work Rate_Low,Attacking Work Rate_Medium,Defensive Work Rate_High,Defensive Work Rate_Low,Defensive Work Rate_Medium
0,91.0,91.0,35.0,169.0,67.0,2190.0,452.0,0.0,4.0,4.0,...,62.0,53.0,62.0,22.0,0.0,1.0,0.0,0.0,1.0,0.0
1,91.0,91.0,34.0,185.0,81.0,2147.0,455.0,1.0,4.0,4.0,...,63.0,58.0,63.0,21.0,0.0,0.0,1.0,0.0,0.0,1.0
2,91.0,91.0,33.0,185.0,81.0,2205.0,458.0,1.0,4.0,4.0,...,64.0,63.0,64.0,22.0,1.0,0.0,0.0,0.0,0.0,1.0
3,91.0,91.0,31.0,181.0,70.0,2303.0,483.0,1.0,5.0,4.0,...,78.0,72.0,78.0,24.0,1.0,0.0,0.0,1.0,0.0,0.0
4,91.0,95.0,23.0,182.0,73.0,2177.0,470.0,1.0,4.0,5.0,...,66.0,57.0,66.0,21.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18415,47.0,56.0,21.0,174.0,68.0,1287.0,274.0,1.0,3.0,2.0,...,40.0,36.0,40.0,15.0,0.0,0.0,1.0,0.0,0.0,1.0
18416,47.0,57.0,17.0,175.0,60.0,1289.0,267.0,1.0,3.0,2.0,...,49.0,49.0,49.0,15.0,0.0,0.0,1.0,0.0,0.0,1.0
18417,47.0,67.0,18.0,170.0,65.0,1333.0,277.0,1.0,3.0,2.0,...,46.0,42.0,46.0,17.0,1.0,0.0,0.0,0.0,0.0,1.0
18418,47.0,61.0,17.0,178.0,65.0,1113.0,226.0,1.0,3.0,2.0,...,47.0,49.0,47.0,15.0,0.0,0.0,1.0,0.0,0.0,1.0


In [11]:
df_mcar_mf

,Overall,Potential,Age,Height(in cm),Weight(in kg),TotalStats,BaseStats,Preferred Foot,Weak Foot Rating,Skill Moves,...,LB Rating,CB Rating,RB Rating,GK Rating,Attacking Work Rate_High,Attacking Work Rate_Low,Attacking Work Rate_Medium,Defensive Work Rate_High,Defensive Work Rate_Low,Defensive Work Rate_Medium
0,91.0,70.0,35.0,169.0,67.0,2190.0,452.0,1.0,4.0,4.0,...,62.0,53.0,64.0,22.0,0.0,1.0,0.0,0.0,1.0,0.0
1,91.0,91.0,22.0,185.0,81.0,2147.0,455.0,1.0,4.0,4.0,...,63.0,58.0,63.0,21.0,0.0,0.0,1.0,0.0,0.0,1.0
2,91.0,91.0,33.0,185.0,81.0,2205.0,458.0,1.0,4.0,4.0,...,64.0,63.0,64.0,22.0,1.0,0.0,0.0,0.0,0.0,1.0
3,91.0,91.0,22.0,181.0,70.0,1620.0,483.0,1.0,5.0,4.0,...,64.0,72.0,78.0,18.0,1.0,0.0,0.0,1.0,0.0,0.0
4,91.0,95.0,23.0,182.0,73.0,2177.0,470.0,1.0,4.0,5.0,...,64.0,57.0,66.0,21.0,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18415,47.0,56.0,21.0,174.0,68.0,1287.0,274.0,1.0,3.0,2.0,...,40.0,36.0,40.0,15.0,0.0,0.0,1.0,0.0,0.0,1.0
18416,47.0,57.0,17.0,175.0,60.0,1289.0,267.0,1.0,3.0,2.0,...,49.0,49.0,49.0,15.0,0.0,0.0,1.0,0.0,0.0,1.0
18417,47.0,67.0,18.0,170.0,65.0,1333.0,277.0,1.0,3.0,2.0,...,46.0,42.0,46.0,17.0,1.0,0.0,0.0,0.0,0.0,1.0
18418,47.0,61.0,17.0,180.0,65.0,1113.0,226.0,1.0,3.0,2.0,...,47.0,49.0,47.0,15.0,0.0,0.0,1.0,0.0,0.0,1.0


In [12]:
df_mnar_mf

,Overall,Potential,Age,Height(in cm),Weight(in kg),TotalStats,BaseStats,Preferred Foot,Weak Foot Rating,Skill Moves,...,LB Rating,CB Rating,RB Rating,GK Rating,Attacking Work Rate_High,Attacking Work Rate_Low,Attacking Work Rate_Medium,Defensive Work Rate_High,Defensive Work Rate_Low,Defensive Work Rate_Medium
0,91.0,0.914894,0.678571,0.588235,0.553571,0.921442,0.820144,1.0,0.75,0.75,...,0.642857,0.652778,0.642857,0.6875,0.0,1.0,0.0,0.0,1.0,1.0
1,91.0,0.914894,0.642857,0.588235,0.571429,0.893754,0.830935,1.0,0.75,0.75,...,0.657143,0.555556,0.657143,0.6875,0.0,0.0,1.0,0.0,0.0,1.0
2,91.0,0.914894,0.607143,0.588235,0.571429,0.931101,0.841727,1.0,0.75,0.75,...,0.671429,0.625000,0.671429,0.6875,1.0,0.0,0.0,0.0,0.0,1.0
3,91.0,0.914894,0.500000,0.509804,0.553571,0.994205,0.931655,1.0,1.00,0.75,...,0.871429,0.750000,0.871429,0.6875,1.0,0.0,0.0,1.0,0.0,0.0
4,91.0,1.000000,0.500000,0.529412,0.553571,0.913071,0.884892,1.0,0.75,1.00,...,0.700000,0.541667,0.700000,0.6875,1.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18415,47.0,0.531915,0.500000,0.588235,0.553571,0.598197,0.579137,1.0,0.50,0.50,...,0.671429,0.652778,0.671429,0.6875,0.0,0.0,1.0,0.0,0.0,1.0
18416,47.0,0.531915,0.500000,0.588235,0.553571,0.598197,0.579137,1.0,0.50,0.50,...,0.671429,0.652778,0.671429,0.6875,0.0,0.0,1.0,0.0,0.0,1.0
18417,47.0,0.404255,0.500000,0.588235,0.553571,0.369607,0.579137,1.0,0.50,0.50,...,0.671429,0.652778,0.671429,0.6875,0.0,0.0,0.0,0.0,0.0,1.0
18418,47.0,0.531915,0.500000,0.588235,0.553571,0.598197,0.579137,1.0,0.50,0.50,...,0.428571,0.652778,0.671429,0.6875,0.0,0.0,1.0,0.0,0.0,1.0


In [13]:
df_mar_knn

,Overall,Potential,Age,Height(in cm),Weight(in kg),TotalStats,BaseStats,Preferred Foot,Weak Foot Rating,Skill Moves,...,LB Rating,CB Rating,RB Rating,GK Rating,Attacking Work Rate_High,Attacking Work Rate_Low,Attacking Work Rate_Medium,Defensive Work Rate_High,Defensive Work Rate_Low,Defensive Work Rate_Medium
0,91.0,91.000000,35.000000,169.000000,67.000000,2190.000000,452.000000,0.000000,4.000000,4.000000,...,62.000000,53.000000,62.000000,22.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
1,91.0,91.000000,34.000000,185.000000,81.000000,2147.000000,455.000000,1.000000,4.000000,4.000000,...,63.000000,58.000000,63.000000,21.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
2,91.0,91.000000,33.000000,185.000000,81.000000,2205.000000,458.000000,1.000000,4.000000,4.000000,...,64.000000,63.000000,64.000000,22.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
3,91.0,91.000000,31.000000,181.000000,70.000000,2303.000000,483.000000,1.000000,5.000000,4.000000,...,78.000000,72.000000,78.000000,24.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
4,91.0,95.000000,23.000000,182.000000,73.000000,2177.000000,470.000000,1.000000,4.000000,5.000000,...,66.000000,57.000000,66.000000,21.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18415,47.0,56.000000,21.000000,174.000000,68.000000,1287.000000,274.000000,1.000000,3.000000,2.000000,...,40.000000,36.000000,40.000000,15.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
18416,47.0,57.000000,17.000000,175.000000,60.000000,1289.000000,267.000000,1.000000,3.000000,2.000000,...,49.000000,49.000000,49.000000,15.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
18417,47.0,67.000000,18.000000,170.000000,65.000000,1333.000000,277.000000,1.000000,3.000000,2.000000,...,46.000000,42.000000,46.000000,17.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
18418,47.0,61.000000,17.000000,178.000000,65.000000,1113.000000,226.000000,1.000000,3.000000,2.000000,...,47.000000,49.000000,47.000000,15.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000


In [14]:
df_mcar_knn

,Overall,Potential,Age,Height(in cm),Weight(in kg),TotalStats,BaseStats,Preferred Foot,Weak Foot Rating,Skill Moves,...,LB Rating,CB Rating,RB Rating,GK Rating,Attacking Work Rate_High,Attacking Work Rate_Low,Attacking Work Rate_Medium,Defensive Work Rate_High,Defensive Work Rate_Low,Defensive Work Rate_Medium
0,91.0,83.854419,35.000000,169.00000,67.0,2190.000000,452.000000,0.608617,4.000000,4.000000,...,62.000000,53.0,67.514465,22.000000,0.0,1.0,0.0,0.000000,1.00000,0.0
1,91.0,91.000000,28.709420,185.00000,81.0,2147.000000,455.000000,1.000000,4.000000,4.000000,...,63.000000,58.0,63.000000,21.000000,0.0,0.0,1.0,0.108217,0.00000,1.0
2,91.0,91.000000,33.000000,185.00000,81.0,2205.000000,458.000000,1.000000,4.000000,4.000000,...,64.000000,63.0,64.000000,22.000000,1.0,0.0,0.0,0.000000,0.00000,1.0
3,91.0,91.000000,28.013928,181.00000,70.0,1600.807955,483.000000,1.000000,5.000000,4.000000,...,68.965962,72.0,78.000000,20.695465,1.0,0.0,0.0,1.000000,0.00000,0.0
4,91.0,95.000000,23.000000,182.00000,73.0,2177.000000,470.000000,0.624670,4.000000,5.000000,...,65.233212,57.0,66.000000,21.000000,1.0,0.0,0.0,0.000000,1.00000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18415,47.0,56.000000,21.000000,174.00000,68.0,1287.000000,274.000000,1.000000,2.714583,2.000000,...,40.000000,36.0,40.000000,15.000000,0.0,0.0,1.0,0.000000,0.00000,1.0
18416,47.0,57.000000,17.000000,175.00000,60.0,1289.000000,267.000000,1.000000,3.000000,2.000000,...,49.000000,49.0,49.000000,15.000000,0.0,0.0,1.0,0.072407,0.00000,1.0
18417,47.0,67.000000,18.000000,170.00000,65.0,1333.000000,277.000000,1.000000,3.000000,2.000000,...,46.000000,42.0,46.000000,17.000000,1.0,0.0,0.0,0.000000,0.10188,1.0
18418,47.0,61.000000,17.000000,184.19424,65.0,1113.000000,226.000000,1.000000,3.000000,2.000000,...,47.000000,49.0,47.000000,15.000000,0.0,0.0,1.0,0.000000,0.00000,1.0


In [15]:
df_mnar_knn

,Overall,Potential,Age,Height(in cm),Weight(in kg),TotalStats,BaseStats,Preferred Foot,Weak Foot Rating,Skill Moves,...,LB Rating,CB Rating,RB Rating,GK Rating,Attacking Work Rate_High,Attacking Work Rate_Low,Attacking Work Rate_Medium,Defensive Work Rate_High,Defensive Work Rate_Low,Defensive Work Rate_Medium
0,91.0,0.914894,0.678571,0.638967,0.622854,0.921442,0.820144,1.0,0.750000,0.750000,...,0.642857,0.741592,0.642857,0.784821,0.394921,1.000000,0.0,0.000000,1.0,0.664638
1,91.0,0.914894,0.642857,0.588235,0.571429,0.893754,0.830935,1.0,0.750000,0.750000,...,0.657143,0.555556,0.657143,0.776864,0.000000,0.000000,1.0,0.000000,0.0,1.000000
2,91.0,0.914894,0.607143,0.638537,0.571429,0.931101,0.841727,1.0,0.750000,0.750000,...,0.671429,0.625000,0.671429,0.785483,1.000000,0.000000,0.0,0.000000,0.0,1.000000
3,91.0,0.914894,0.566209,0.509804,0.614647,0.994205,0.931655,1.0,1.000000,0.750000,...,0.871429,0.750000,0.871429,0.777518,1.000000,0.027233,0.0,1.000000,0.0,0.000000
4,91.0,1.000000,0.562694,0.529412,0.622471,0.913071,0.884892,1.0,0.750000,1.000000,...,0.700000,0.541667,0.700000,0.788486,1.000000,0.000000,0.0,0.000000,1.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18415,47.0,0.569621,0.559665,0.582940,0.550606,0.616652,0.583561,1.0,0.500000,0.566919,...,0.635362,0.623269,0.631050,0.660507,0.000000,0.000000,1.0,0.013013,0.0,1.000000
18416,47.0,0.569346,0.559743,0.582527,0.545149,0.608146,0.583561,1.0,0.555102,0.566919,...,0.627027,0.616714,0.630020,0.659821,0.000000,0.000000,1.0,0.000000,0.0,0.940943
18417,47.0,0.404255,0.559471,0.585769,0.550645,0.369607,0.583561,1.0,0.500000,0.566919,...,0.631262,0.629366,0.631797,0.659915,0.072299,0.000000,0.0,0.000000,0.0,1.000000
18418,47.0,0.569473,0.559895,0.583145,0.544738,0.598681,0.583561,1.0,0.500000,0.566919,...,0.428571,0.608748,0.622384,0.660109,0.000000,0.000000,1.0,0.000000,0.0,0.948513


In [16]:
df_mar_mice

,Overall,Potential,Age,Height(in cm),Weight(in kg),TotalStats,BaseStats,Preferred Foot,Weak Foot Rating,Skill Moves,...,LB Rating,CB Rating,RB Rating,GK Rating,Attacking Work Rate_High,Attacking Work Rate_Low,Attacking Work Rate_Medium,Defensive Work Rate_High,Defensive Work Rate_Low,Defensive Work Rate_Medium
0,91.0,91.000000,35.000000,169.00000,67.000000,2190.000000,452.000000,0.00000,4.000000,4.000000,...,62.000000,53.000000,62.000000,22.000000,0.000000,1.000000,0.000000,0.000000,1.000000,0.000000
1,91.0,91.000000,34.000000,185.00000,81.000000,2147.000000,455.000000,1.00000,4.000000,4.000000,...,63.000000,58.000000,63.000000,21.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
2,91.0,91.000000,33.000000,185.00000,81.000000,2205.000000,458.000000,1.00000,4.000000,4.000000,...,64.000000,63.000000,64.000000,22.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
3,91.0,91.000000,31.000000,181.00000,70.000000,2303.000000,483.000000,1.00000,5.000000,4.000000,...,78.000000,72.000000,78.000000,24.000000,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000
4,91.0,95.000000,23.000000,182.00000,73.000000,2177.000000,470.000000,1.00000,4.000000,5.000000,...,66.000000,57.000000,66.000000,21.000000,1.000000,0.000000,0.000000,0.000000,1.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18415,47.0,56.000000,21.000000,174.00000,68.000000,1287.000000,274.000000,1.00000,3.000000,2.000000,...,40.000000,36.000000,40.000000,15.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
18416,47.0,57.000000,17.000000,175.00000,60.000000,1289.000000,267.000000,1.00000,3.000000,2.000000,...,49.000000,49.000000,49.000000,15.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000
18417,47.0,67.000000,18.000000,170.00000,65.000000,1333.000000,277.000000,1.00000,3.000000,2.000000,...,46.000000,42.000000,46.000000,17.000000,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000
18418,47.0,61.000000,17.000000,178.00000,65.000000,1113.000000,226.000000,1.00000,3.000000,2.000000,...,47.000000,49.000000,47.000000,15.000000,0.000000,0.000000,1.000000,0.000000,0.000000,1.000000


In [17]:
df_mcar_mice

,Overall,Potential,Age,Height(in cm),Weight(in kg),TotalStats,BaseStats,Preferred Foot,Weak Foot Rating,Skill Moves,...,LB Rating,CB Rating,RB Rating,GK Rating,Attacking Work Rate_High,Attacking Work Rate_Low,Attacking Work Rate_Medium,Defensive Work Rate_High,Defensive Work Rate_Low,Defensive Work Rate_Medium
0,91.0,86.967876,35.000000,169.000000,67.0,2190.000000,452.000000,0.835710,4.000000,4.000000,...,62.0,53.0,62.0,22.000000,0.0,1.0,0.0,0.000000e+00,1.000000e+00,0.0
1,91.0,91.000000,30.116887,185.000000,81.0,2147.000000,455.000000,1.000000,4.000000,4.000000,...,63.0,58.0,63.0,21.000000,0.0,0.0,1.0,1.501006e-09,0.000000e+00,1.0
2,91.0,91.000000,33.000000,185.000000,81.0,2205.000000,458.000000,1.000000,4.000000,4.000000,...,64.0,63.0,64.0,22.000000,1.0,0.0,0.0,0.000000e+00,0.000000e+00,1.0
3,91.0,91.000000,30.858509,181.000000,70.0,2306.426461,483.000000,1.000000,5.000000,4.000000,...,78.0,72.0,78.0,23.451601,1.0,0.0,0.0,1.000000e+00,0.000000e+00,0.0
4,91.0,95.000000,23.000000,182.000000,73.0,2177.000000,470.000000,0.737998,4.000000,5.000000,...,66.0,57.0,66.0,21.000000,1.0,0.0,0.0,0.000000e+00,1.000000e+00,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18415,47.0,56.000000,21.000000,174.000000,68.0,1287.000000,274.000000,1.000000,2.926906,2.000000,...,40.0,36.0,40.0,15.000000,0.0,0.0,1.0,0.000000e+00,0.000000e+00,1.0
18416,47.0,57.000000,17.000000,175.000000,60.0,1289.000000,267.000000,1.000000,3.000000,2.000000,...,49.0,49.0,49.0,15.000000,0.0,0.0,1.0,8.484924e-11,0.000000e+00,1.0
18417,47.0,67.000000,18.000000,170.000000,65.0,1333.000000,277.000000,1.000000,3.000000,2.000000,...,46.0,42.0,46.0,17.000000,1.0,0.0,0.0,0.000000e+00,4.052754e-09,1.0
18418,47.0,61.000000,17.000000,168.157022,65.0,1113.000000,226.000000,1.000000,3.000000,2.000000,...,47.0,49.0,47.0,15.000000,0.0,0.0,1.0,0.000000e+00,0.000000e+00,1.0


In [18]:
df_mnar_mice

,Overall,Potential,Age,Height(in cm),Weight(in kg),TotalStats,BaseStats,Preferred Foot,Weak Foot Rating,Skill Moves,...,LB Rating,CB Rating,RB Rating,GK Rating,Attacking Work Rate_High,Attacking Work Rate_Low,Attacking Work Rate_Medium,Defensive Work Rate_High,Defensive Work Rate_Low,Defensive Work Rate_Medium
0,91.0,0.914894,0.678571,0.469440,0.615822,0.921442,0.820144,1.0,0.750000,0.750000,...,0.642857,0.496852,0.642857,0.696634,-0.025198,1.000000,0.0,0.000000,1.0,-0.011552
1,91.0,0.914894,0.642857,0.588235,0.571429,0.893754,0.830935,1.0,0.750000,0.750000,...,0.657143,0.555556,0.657143,0.696634,0.000000,0.000000,1.0,0.000000,0.0,1.000000
2,91.0,0.914894,0.607143,0.454534,0.571429,0.931101,0.841727,1.0,0.750000,0.750000,...,0.671429,0.625000,0.671429,0.696634,1.000000,0.000000,0.0,0.000000,0.0,1.000000
3,91.0,0.914894,0.542449,0.509804,0.517303,0.994205,0.931655,1.0,1.000000,0.750000,...,0.871429,0.750000,0.871429,0.696634,1.000000,-0.051863,0.0,1.000000,0.0,0.000000
4,91.0,1.000000,0.543483,0.529412,0.515563,0.913071,0.884892,1.0,0.750000,1.000000,...,0.700000,0.541667,0.700000,0.696634,1.000000,0.000000,0.0,0.000000,1.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18415,47.0,0.469385,0.494995,0.664328,0.525295,0.343172,0.255879,1.0,0.500000,0.520044,...,0.487469,0.434055,0.465369,0.696634,0.000000,0.000000,1.0,-0.031007,0.0,1.000000
18416,47.0,0.381434,0.586743,0.433405,0.468838,0.365642,0.286388,1.0,0.562206,0.549020,...,0.470682,0.458692,0.460511,0.696634,0.000000,0.000000,1.0,0.000000,0.0,0.997339
18417,47.0,0.404255,0.526749,0.676843,0.544520,0.369607,0.225270,1.0,0.500000,0.424177,...,0.468487,0.410092,0.452309,0.696634,1.001116,0.000000,0.0,0.000000,0.0,1.000000
18418,47.0,0.312932,0.620314,0.610727,0.550960,0.403849,0.272402,1.0,0.500000,0.471509,...,0.428571,0.415201,0.410204,0.696634,0.000000,0.000000,1.0,0.000000,0.0,0.999465


In [19]:
model = LGBMRegressor()

X_train, X_test, y_train, y_test = train_test_split(df_mar.drop(columns=["Overall"]), df_mar["Overall"], test_size=0.2)

model.fit(X_train, y_train)
pred = model.predict(X_test)

mae = mean_absolute_error(y_test, pred)

print(mae)


0.8986260272473886


In [20]:
results = {}

datasets = [df, df_mar, df_mcar, df_mnar,
            df_mar_mean, df_mcar_mean, df_mnar_mean,
            df_mar_mf, df_mcar_mf, df_mnar_mf,
            df_mar_knn, df_mcar_knn, df_mnar_knn,
            df_mar_mice, df_mcar_mice, df_mnar_mice]

names = ["Completo", "MAR", "MCAR", "MNAR",
         "MAR Mean Imputation", "MCAR Mean Imputation", "MNAR Mean Imputation",
         "MAR Most Frequent Imputation", "MCAR Most Frequent Imputation", "MNAR Most Frequent Imputation",
         "MAR KNN Imputation", "MCAR KNN Imputation", "MNAR KNN Imputation",
         "MAR MICE Imputation", "MCAR MICE Imputation", "MNAR MICE Imputation"]

models = [XGBRegressor(verbosity=0, missing=-1), LGBMRegressor()]

for dataset, name in zip(datasets, names):
    X_train, X_test, y_train, y_test = train_test_split(dataset.drop(columns=["Overall"]), dataset["Overall"], test_size=0.2)

    for model in models:
        model.fit(X_train, y_train)

        pred = model.predict(X_test)

        mae = mean_absolute_error(y_test, pred)
        mse = mean_squared_error(y_test, pred)
        r2 = r2_score(y_test, pred)

        results[type(model).__name__ + " - " + name] = {"MAE": mae, "MSE": mse, "R2 Score": r2}

print(results)

{'XGBRegressor - Completo': {'MAE': 0.6284836449141611, 'MSE': 0.6895982106926152, 'R2 Score': 0.9848633159842713}, 'LGBMRegressor - Completo': {'MAE': 0.5593090005347873, 'MSE': 0.5902962873970046, 'R2 Score': 0.9870429936745166}, 'XGBRegressor - MAR': {'MAE': 0.9032722995024939, 'MSE': 3.041024365704871, 'R2 Score': 0.9335173402057746}, 'LGBMRegressor - MAR': {'MAE': 0.8291007697486004, 'MSE': 2.9098593014784297, 'R2 Score': 0.9363848615713368}, 'XGBRegressor - MCAR': {'MAE': 0.8188374495532174, 'MSE': 1.150911464741879, 'R2 Score': 0.9756685789716076}, 'LGBMRegressor - MCAR': {'MAE': 0.7311581589807641, 'MSE': 0.9646659159754544, 'R2 Score': 0.979605996401641}, 'XGBRegressor - MNAR': {'MAE': 0.8075880709222552, 'MSE': 1.1564367029890181, 'R2 Score': 0.9747479171094826}, 'LGBMRegressor - MNAR': {'MAE': 0.6915064239862926, 'MSE': 0.8936842857409615, 'R2 Score': 0.9804854086668521}, 'XGBRegressor - MAR Mean Imputation': {'MAE': 0.9312014284662005, 'MSE': 3.5228751011844404, 'R2 Score':

In [21]:
for key, value in results.items():
    print(key, ' : ', value)

XGBRegressor - Completo  :  {'MAE': 0.6284836449141611, 'MSE': 0.6895982106926152, 'R2 Score': 0.9848633159842713}
LGBMRegressor - Completo  :  {'MAE': 0.5593090005347873, 'MSE': 0.5902962873970046, 'R2 Score': 0.9870429936745166}
XGBRegressor - MAR  :  {'MAE': 0.9032722995024939, 'MSE': 3.041024365704871, 'R2 Score': 0.9335173402057746}
LGBMRegressor - MAR  :  {'MAE': 0.8291007697486004, 'MSE': 2.9098593014784297, 'R2 Score': 0.9363848615713368}
XGBRegressor - MCAR  :  {'MAE': 0.8188374495532174, 'MSE': 1.150911464741879, 'R2 Score': 0.9756685789716076}
LGBMRegressor - MCAR  :  {'MAE': 0.7311581589807641, 'MSE': 0.9646659159754544, 'R2 Score': 0.979605996401641}
XGBRegressor - MNAR  :  {'MAE': 0.8075880709222552, 'MSE': 1.1564367029890181, 'R2 Score': 0.9747479171094826}
LGBMRegressor - MNAR  :  {'MAE': 0.6915064239862926, 'MSE': 0.8936842857409615, 'R2 Score': 0.9804854086668521}
XGBRegressor - MAR Mean Imputation  :  {'MAE': 0.9312014284662005, 'MSE': 3.5228751011844404, 'R2 Score':

### **NORMAL REGRESSION MODELS**

In [23]:
datasets = [df,
            df_mar_mean, df_mcar_mean, df_mnar_mean,
            df_mar_mf, df_mcar_mf, df_mnar_mf,
            df_mar_knn, df_mcar_knn, df_mnar_knn,
            df_mar_mice, df_mcar_mice, df_mnar_mice]

names = ["Completo",
         "MAR Mean Imputation", "MCAR Mean Imputation", "MNAR Mean Imputation",
         "MAR Most Frequent Imputation", "MCAR Most Frequent Imputation", "MNAR Most Frequent Imputation",
         "MAR KNN Imputation", "MCAR KNN Imputation", "MNAR KNN Imputation",
         "MAR MICE Imputation", "MCAR MICE Imputation", "MNAR MICE Imputation"]

models = [GaussianNB(), LinearRegression(), RandomForestRegressor(), KNeighborsRegressor(), DecisionTreeRegressor()]

for dataset, name in zip(datasets, names):
    X_train, X_test, y_train, y_test = train_test_split(dataset.drop(columns=["Overall"]), dataset["Overall"], test_size=0.2)

    for model in models:
        model.fit(X_train, y_train)

        pred = model.predict(X_test)

        mae = mean_absolute_error(y_test, pred)
        mse = mean_squared_error(y_test, pred)
        r2 = r2_score(y_test, pred)

        results[type(model).__name__ + " - " + name] = {"MAE": mae, "MSE": mse, "R2 Score": r2}

for key, value in results.items():
    print(key, ' : ', value)

XGBRegressor - Completo  :  {'MAE': 0.6284836449141611, 'MSE': 0.6895982106926152, 'R2 Score': 0.9848633159842713}
LGBMRegressor - Completo  :  {'MAE': 0.5593090005347873, 'MSE': 0.5902962873970046, 'R2 Score': 0.9870429936745166}
XGBRegressor - MAR  :  {'MAE': 0.9032722995024939, 'MSE': 3.041024365704871, 'R2 Score': 0.9335173402057746}
LGBMRegressor - MAR  :  {'MAE': 0.8291007697486004, 'MSE': 2.9098593014784297, 'R2 Score': 0.9363848615713368}
XGBRegressor - MCAR  :  {'MAE': 0.8188374495532174, 'MSE': 1.150911464741879, 'R2 Score': 0.9756685789716076}
LGBMRegressor - MCAR  :  {'MAE': 0.7311581589807641, 'MSE': 0.9646659159754544, 'R2 Score': 0.979605996401641}
XGBRegressor - MNAR  :  {'MAE': 0.8075880709222552, 'MSE': 1.1564367029890181, 'R2 Score': 0.9747479171094826}
LGBMRegressor - MNAR  :  {'MAE': 0.6915064239862926, 'MSE': 0.8936842857409615, 'R2 Score': 0.9804854086668521}
XGBRegressor - MAR Mean Imputation  :  {'MAE': 0.9312014284662005, 'MSE': 3.5228751011844404, 'R2 Score':